# Cleaning
Welcome to the cleaning code. In this Notebook you can go thorugh the cleaning of the data, mostly I used some functions, using the help() function you can acces an explanation of each of them, if you want to go through the code of each function, visit the file function.py in Code directory in the repository. 

In [1]:
# First lets import all the libraries we are gonna use,
import functions as fun 
import pandas as pd


In [2]:
# Now lets open and define the DataFrame,
df = pd.read_csv('/mnt/c/Users/marcn/Documents/Ironhack/sharky/Data/attacks.csv', encoding="ISO-8859-1")
new_dict = {i: i.strip().replace(" ", "_").replace('.','').lower() for i in df.columns}
df.rename(columns=new_dict, inplace=True)
df.drop(columns= ['unnamed:_22', 'unnamed:_23','area','activity','location', 'type','injury', 'fatal_(y/n)', 'time','investigator_or_source', 'pdf', 'href_formula', 'href','original_order' ], inplace=True) # We are dropping the unnecesary data for our study
df.dropna(axis = 0, how = 'all', inplace = True)
# Let's make it a little more easy to work with by renaming columns:
df.drop_duplicates()

,case_number,date,year,country,name,sex,age,species,case_number1,case_number2
0,2018.06.25,25-Jun-2018,2018.0,USA,Julie Wolfe,F,57,White shark,2018.06.25,2018.06.25
1,2018.06.18,18-Jun-2018,2018.0,USA,Adyson McNeely,F,11,NaN,2018.06.18,2018.06.18
2,2018.06.09,09-Jun-2018,2018.0,USA,John Denges,M,48,NaN,2018.06.09,2018.06.09
3,2018.06.08,08-Jun-2018,2018.0,AUSTRALIA,male,M,NaN,2 m shark,2018.06.08,2018.06.08
4,2018.06.04,04-Jun-2018,2018.0,MEXICO,Gustavo Ramos,M,NaN,"Tiger shark, 3m",2018.06.04,2018.06.04
...,...,...,...,...,...,...,...,...,...,...
6299,ND.0003,1900-1905,0.0,USA,Coast Guard personnel,M,NaN,NaN,ND.0003,ND.0003
6300,ND.0002,1883-1889,0.0,PANAMA,Jules Patterson,M,NaN,NaN,ND.0002,ND.0002
6301,ND.0001,1845-1853,0.0,CEYLON (SRI LANKA),male,M,15,NaN,ND.0001,ND.0001
6302,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
# Once with our dataframe defined we can start cleaning the data, lets first do a little further exploration on it,
# Let's see what data contains information about the Year when the attack happened,
a = list(df.columns)
print(a)

['case_number', 'date', 'year', 'country', 'name', 'sex', 'age', 'species', 'case_number1', 'case_number2']


In [4]:
# Let's check the nulls,
df.isnull().sum() # It is quite obvious we can drop the last 2 columns

case_number        1
date            2401
year            2403
country         2451
name            2611
sex             2966
age             5232
species         5239
case_number1    2401
case_number2    2401
dtype: int64

In [5]:
#some important observations:
""" We see that many cases are not dated, just labeld as before some year. We can't use this data, it is not precise.
Also there is some data that is labeled with a range between 2 years. The range is never higher than 20 years in the observed sample, so i am going
to use this data, eventhough it is not very precise it is bounded. """
df[df[['case_number','date','year']].notnull().all(1)].sample(3)


,case_number,date,year,country,name,sex,age,species,case_number1,case_number2
1060,2009.12.22,22-Dec-2009,2009.0,MOZAMBIQUE,Peter Fraser,M,27,"Zambesi shark, 2m",2009.12.22,2009.12.22
5666,1893.06.22,22-Jun-1893,1893.0,LEBANON,crew,M,NaN,NaN,1893.06.22,1893.06.22
5367,1912.08.30,30-Aug-1912,1912.0,USA,Edward Coffee,M,12,NaN,1912.08.30,1912.08.30


In [6]:
df.loc[df['year'].isnull()].tail()


,case_number,date,year,country,name,sex,age,species,case_number1,case_number2
8698,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8699,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8700,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8701,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25722,xx,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Cleaning by date,
The feature I want to focus on is the year of the attack, so what I am going to do is try to assing a year to each attack, if possible. First, I observed that 125 of the attacks are assigned year '0.0', but for most of the cases in the date column there is more information about the date, all that have a date with the string 'Before' in it are not going to count for the study, and the ones with a range in them are going to be assigned the date in the middle of the range. Also, I am going to take in account all the attacks that happened A.D, and drop everything B.C. 

The order of cleaning will go as follows:
1. First i am going locate all the data that has a NaN or a 0.0 in the Year column. And work with those.
2. After that, I am going to grop all the rows with Dates that have 'B.C' in them.
3. Now, the Dates that go as: '\d{4}-\d{4}' are going to be replaced with the year in between. 
4. In some rare cases there is also a string such as: Late 1970's, we are going to try and find an s afer a number, if we find it we are going to return number+5 (in this cas 1975, so it is more or less precise with both earlies and late).
5. And finally, for the rest with a NaN or a 0.0 in their year column, I am going to try and find a string with '\d{4}|\d{3}' in them, and replace the year value for the found value. 



In [7]:
df.case_number.sample(10)

4879      1936.06.06
677     2013.05.08.a
4961      1933.06.21
1467      2006.06.27
2427    1994.07.09.c
4058      1959.09.27
4177    1958.00.00.d
6513               0
4673      1943.05.27
2597    1991.06.07.b
Name: case_number, dtype: object

In [8]:
df['year'] = df.year.apply(fun.clean_column)
df['case_number'] = df.case_number.apply(fun.clean_column)
df['case_number1'] = df.case_number1.apply(fun.clean_column)
df['case_number2'] = df.case_number2.apply(fun.clean_column)
df['date'] = df.date.apply(fun.clean_column)
df.dropna(axis = 0, how = 'all', inplace = True)

In [9]:
df[['case_number','date' ,'year', 'case_number1', 'case_number2']]

,case_number,date,year,case_number1,case_number2
0,2018,2018,2018,2018,2018
1,2018,2018,2018,2018,2018
2,2018,2018,2018,2018,2018
3,2018,2018,2018,2018,2018
4,2018,2018,2018,2018,2018
...,...,...,...,...,...
6297,0005,1903,NaN,0005,0005
6298,0004,1903,NaN,0004,0004
6299,0003,1900,NaN,0003,0003
6300,0002,1883,NaN,0002,0002


In [10]:
df.year = fun.year_replacement(df)

In [11]:
# Let's now clean the country column and make it more easy to read:
df['country'], df['continent'], errors = fun.country_clean(df['country'])
#df.country.unique()

In [12]:
# let's clean the data that yields information about the people that were attacked by sharks,
df['name'] = df['name'].apply(fun.clean_names)
df.age = df.age.apply(fun.clean_age)
df.sex = df.sex.apply(fun.clean_sex)
df.columns

Index(['case_number', 'date', 'year', 'country', 'name', 'sex', 'age',
       'species', 'case_number1', 'case_number2', 'continent'],
      dtype='object')

Now that we have cleaned the data it is importat that we store it in a new file, so then we can read it and create the figures to visualize the data. But first I am going to drop some columns we might not need anymore. 

In [13]:
new_df = df.drop(columns= ['case_number', 'date','case_number1', 'case_number2']) # We are dropping the unnecesary data for our study
new_df.dropna(axis = 0, how = 'all', inplace = True)
new_df.shape # Notice that quantity of data has been drasticly reduced, that is because the quality of the data wasn't very good. 
# Also, now we must have 6 columns: year, country, name, sex, age and species.

(6302, 7)

In [14]:
new_df.to_csv('../Data/clean_attacks.csv')